# Exercise 16: Sentiment Analysis with NLTK

<sup>This lab is based on https://github.com/lesley2958/twilio-sent-analysis</sup>


Welcome to exercise 16!

In this exercise, you will step through two simple examples using scikit-learn and *NLTK*, one of the most popular Natural Language Processing and Analysis libraries for the Python programming language. NLTK features various lexical analyses, n-gram and collocation identification, part-of-speech tagging and named-entity recognition amongst others.

<img src="images/nltk_banner.jpeg" />

If you have launched this notebook in `binder` the `nltk` library should already have been installed on the underlying virtual machine that the copy of Jupyter Notebook is running in. If you have downloaded this notebook to use on your own computer (or in the CTR), you might need to install scikit-learn first before running this notebook. To install scikit-learn, open up the command-line (Command prompt on Windows, or Git Bash in the CTR) and type:

```
conda install nltk
```

and press Enter. Some text should whiz by indicating that it's installing various things. Once that is done, please restart Jupyter Notebook.

For full documentation about how to use NLTK you can reference here: https://www.nltk.org

In this lab session, we will try out some *sentiment analysis* on Twitter data. So you might be asking, what exactly is "sentiment analysis"? 

Well, it's exactly what it sounds like: it's building a computational system to determine the emotional tone behind words. This is important because it allows you to gain an understanding of the attitudes, opinions, and emotions of the people in your data. At a higher level, sentiment analysis involves Natural language processing and artificial intelligence by taking the actual text element, transforming it into a format that a machine can read, and using statistics to determine the actual sentiment.

In this tutorial, we'll review some of the methods used to determine sentiment. To use NLTK in Python code, you import from `nltk`.

Sentiment Analysis isn't a new problem. There are thousands of labeled data out there, labels varying from simple positive and negative to more complex systems that determine *how* positive or negative is a given text. With that said, I've selected a pre-labeled set of data consisting of tweets from Twitter. Using this data, we'll begin by building a sentiment analysis model with scikit-learn. 

## Part A: Building a sentiment analysis model

In this tutorial, we'll specifically use the Logistic Regression model from scikit-learn, which is a linear model commonly used for classifying binary data. The first part quite straightforward - I just want you to step through the code (there's nothing to solve in the first part) to get a feel for what is going on. In the second part, I want you to apply the sentiment analysis methods here to your own data.

### Preparing the Data

To implement our linear classifier, we need the twitter data in a format that allows us to feed it into the classifier. Using the `sklearn.feature_extraction.text.CountVectorizer`, we will convert the text documents to a matrix of token counts. As you'll see soon enough, these vector counts will be what the classifier will ultimately use to train. So first, we import all the needed modules: 

In [1]:
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer        

But before we even get to vectorizing, we haven't even read in our data! Each file is a text file where each line is a sentence, so we can use the built-in `open()` function to split the file into tweets and append them to lists. Note that text and labels have to be separated for the test phase of this example!

In [5]:
fp = open("neg_tweets.txt")

In [6]:
fp.readlines()[:20]

['" @iggigg too busy to see me in London this evening. What is a boy to do?"\n',
 '" cavs lost, and I got this sinking feeling we are going to lose Lebron in 2010 also...why must my home city SUCK? Ah well, LETS GO BROWNS!"\n',
 '" the closest BGT tour is Cardiff or London dam it why doesnt anybody other than the Chuckle Brothers tour the Westcountry?"\n',
 '" Why do other pet care people try to run others out of business? Or send suspicious e-mails fishing for info?"\n',
 '"-gasps- dananananaykroyd touring aussie ....  GRRR WHY ALWAYS SYDNEY AND MELBZ FFS"\n',
 '"....If i am going warsal that means no church in the morning if i dont go church but i think it imporant to see my nan coz she go cancer "\n',
 '"....wow the CAVS are sleeping ! I hate these blowout games "\n',
 '"...no more Lebron "\n',
 '"\'adawiyah is having fever.  hope the other 2 are not."\n',
 '"[-O] I wish the frogs weren\'t becoming extinct    http://tinyurl.com/cxe8w7"\n',
 '"@ Starbucks Windward Mall lovin the AC. 

In [8]:
text_pos = []
labels_pos = []
with open("pos_tweets.txt") as f:
    for i in f: 
        text_pos.append(i) 
        labels_pos.append('pos')

text_neg = []
labels_neg = []
with open("neg_tweets.txt") as f:
    for i in f: 
        text_neg.append(i)
        labels_neg.append('neg')

Now that the data is properly stored in Python lists, it's time to split this data into training and test data. For the purpose of this tutorial, we'll keep 80% for training and 20% for testing -- we do this with Python list slicing! 

In [9]:
training_text = text_pos[:int((.8)*len(text_pos))] + text_neg[:int((.8)*len(text_neg))]
training_labels = labels_pos[:int((.8)*len(labels_pos))] + labels_neg[:int((.8)*len(labels_neg))]

test_text = text_pos[int((.8)*len(text_pos)):] + text_neg[int((.8)*len(text_neg)):]
test_labels = labels_pos[int((.8)*len(labels_pos)):] + labels_neg[int((.8)*len(labels_neg)):]

Next, we initialize a vectoriser with the CountVectorizer class. Because we haven't pre-processed any of the data, we'll set lowercase to `False` and exclude stop word removal or stemming. 

In [10]:
vectorizer = CountVectorizer(
    analyzer = 'word',
    lowercase = False,
    max_features = 85
)

Next, we use the `fit_transform()` method to transform our corpus data into feature vectors. Since the input needed is a list of strings, we concatenate all of our training and test data. 

In [11]:
features = vectorizer.fit_transform(
    training_text + test_text)

features_nd = features.toarray() # for easy use

As a final step, we'll split the training data to get an evaluation set through scikit-learn's built-in cross validation method. All we need to do is provide the data and assign a training percentage (in this case, 80%).

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(
        features_nd[0:len(training_text)], 
        training_labels,
        train_size=0.80, 
        random_state=1234)

/Users/davidjohnson/Development/ox-p4ds-mt18/.venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2069: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### Linear Classifier

Finally, we can build the classifier for this corpus! As mentioned before, we'll be using the Logistic Regression from scikit-learn, so we'll start there: 

In [13]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()

Once the model is initialized, we have to fit it to our specific dataset, so we use scikit-learn's `fit()` method to do so:

In [14]:
log_model = log_model.fit(X=X_train, y=y_train)

/Users/davidjohnson/Development/ox-p4ds-mt18/.venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


And finally, we use this classifier to label the evaluation set we created earlier:

In [15]:
test_pred = log_model.predict(X_test)

### Accuracy

Now just for our own "fun", let's take a look at some of the classifications our model makes! We'll choose a random set and then call our model on each! 

In [17]:
import random
spl = random.sample(range(len(test_pred)), 10)
for text, sentiment in zip(test_text, test_pred[spl]):
    print(sentiment, text)

pos "Love that the Obamas are bringing back date night. Men, take note  http://bit.ly/ebPBZ"

neg "loves chocolate milk  and that is GF YEAH.."

neg "loving life... and loving you "

neg "Lunch date with @Londonmitch to go to Leathenhall Market's cheese shop was really nice "

neg "Made my evening: Starbucks barista complimented me on my hamsa scarf. When he heard I designed it and sell them on etsy he was "

neg "Matt came to visit!  canton with him for dinner then prolly stopping somewhere for drinks and the cavs "

pos "May be going to London next week "

neg "me and rosa are going 2 start our own youtube channel! i'm super excited that we get 2 share our RaNdOmNeSs with the world!!!! "

neg "Modern warfare 2 gameplay looks goood  #xboxe3"

neg "Momz just made it back from Vegas, yayyyyy! "



Just glancing over the examples above, it's pretty obvious there are some misclassifications. But obviously we want to do more than just 'eyeball' the data, so let's actually calculate the accuracy score. 

What's the point in creating a machine learning algorithm if you have no idea how it performs? This is why we left some of the dataset for testing purposes. In scikit-learn, there is a function called sklearn.metrics.accuracy_score which calculates the accuracy percentage. Using this, we see that this model has an accuracy of about 76%. 

In [19]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, test_pred))

0.7601246105919003


Yikes. 76% is better than randomly guessing, but still pretty low as far as classification accuracy goes. *Maybe* this is the best we can do with this dataset, but maybe we *can* do better? So let's give it a try with the Python module `nltk`. 

### Preparing the Data 

`nltk` is a much different from `scikit-learn`. `nltk` specializes and is *made for* natural language processing tasks, so needless to say, it was expected that `scikit-learn` wouldn't necessarily be the best choice. 
 
With that said, we'll now use `nltk` to build a sentiment analysis model on the same dataset. `nltk` requires a different data format, which is why I've implemented the function below:

In [20]:
import nltk
nltk.download('punkt')

def format_sentence(sent):
    return({word: True for word in nltk.word_tokenize(sent)})

print(format_sentence("My cat is very cute"))

{'My': True, 'cat': True, 'is': True, 'very': True, 'cute': True}


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/davidjohnson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Here, `format_sentence` changes each tweet into a dictionary of words mapped to `True` booleans. Though not obvious from this function alone, this will eventually allow us to train our prediction model by splitting the text into its tokens, i.e. <i>tokenizing</i> the text. You'll learn about why this format is important in a later section.

Using the data on the github repo, we'll actually format the positively and negatively labeled data.

In [21]:
pos = []
with open("pos_tweets.txt") as f:
    for i in f: 
        pos.append([format_sentence(i), 'pos'])

In [22]:
neg = []
with open("neg_tweets.txt") as f:
    for i in f: 
        neg.append([format_sentence(i), 'neg'])

Next, we'll split the labeled data into the training and test data, just as we did before.

In [23]:
training = pos[:int((.8)*len(pos))] + neg[:int((.8)*len(neg))]
test = pos[int((.8)*len(pos)):] + neg[int((.8)*len(neg)):]

### Building a Classifier

All NLTK classifiers work with feature structures, which can be simple dictionaries mapping a feature name to a feature value. In this example, we use the Naive Bayes Classifier, which makes predictions based on the word frequencies associated with each label of positive or negative.

In [24]:
from nltk.classify import NaiveBayesClassifier

classifier = NaiveBayesClassifier.train(training)

Since the Naive Bayes Classifier is based entirely off of the frequencies associated with each label for a given word, we can call a function `show_most_informative_features()` to see which words are the highest indicators of a positive or negative label:

In [25]:
classifier.show_most_informative_features()

Most Informative Features
                      no = True              neg : pos    =     19.4 : 1.0
                    love = True              pos : neg    =     19.0 : 1.0
                 awesome = True              pos : neg    =     17.2 : 1.0
                headache = True              neg : pos    =     16.2 : 1.0
                      Hi = True              pos : neg    =     12.7 : 1.0
                   Thank = True              pos : neg    =      9.7 : 1.0
                     fan = True              pos : neg    =      9.7 : 1.0
               beautiful = True              pos : neg    =      9.7 : 1.0
                     New = True              pos : neg    =      9.7 : 1.0
                    haha = True              pos : neg    =      9.3 : 1.0


Notice that there are three columns. Column 1 is why we used `format_sentence()` to map each word to a `True` value. What it does is count the number of occurences of each word for both labels to compute the ratio between the two, which is what column 3 represents. Column 2 lets us know which label occurs more frequently (the label on the left is the label most associated with the corresponding word. 

### Classification

Just to see how our model works, let's try the classifier out with a positive example: 

In [27]:
example1 = "Uppsala University is great!"
print(classifier.classify(format_sentence(example1)))

pos


Now let's try out an example we'd expect a negative label: 

In [28]:
example2 = "I'm sad that as an early career academic I don't get a better pension deal!"
print(classifier.classify(format_sentence(example2)))

neg


So what happens when we mix words of different sentiment labels? Let's take a look at this example:

In [30]:
example3 = "I have no worries!"
print(classifier.classify(format_sentence(example3)))

neg


And we've found a mislabel! Naive Bayes doesn't consider the relationship *between* words, which is why it wasn't able to catch the fact that "no" acted as a negator to the word `headache`. Instead, it read two negative indicators and classified it as such. 

Given that, we can probably expect a less than perfect accuracy rate.

### Accuracy

Just like `scikit-learn`, nltk has a built in method that computes the accuracy rate of our model:

In [31]:
from nltk.classify.util import accuracy
print(accuracy(classifier, test))

0.8308457711442786


And we have improved performance! Awesome. Now, 83% is fairly solid, but it could be better! If you look at the actual data, you'll see that the data is kind of messy - there are typos, abbreviations, grammatical errors of all sorts.

So how do we handle that? Can we handle that? We did some of that with our Project Gutenberg examples in the earlier weeks - stripping out punctuation, and making the input data case-insensitive. We can also filter out word that are not of interest, such as "and", "the", "via" and so on. These are called *stopwords*. `nltk` allows us to get a corpus of commonly found stopwords, which we can then use to filter out of our data.

In [32]:
import numpy as np
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
punctuation = list(string.punctuation)
english_stopwords = stopwords.words('english')
print("Punctuation: ", punctuation)
print("Stopwords: ", english_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/davidjohnson/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Punctuation:  ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
Stopwords:  ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'onc

Try and apply these stoplists to what we did earlier to see if it makes any difference to the performance of our models. 

What other filters can we apply if we are dealing with the [anatomy of tweets](https://marketing.wtwhmedia.com/twitter-for-business-101-the-anatomy-of-a-tweet/) from Twitter?

## Part B: Applying using NLTK with your own data

Now you have an idea of how to use NLTK, try and apply some natural language processing to the tweets you gathered in Lab 8 (the `tweepy` exercise). Some ideas of what you could try to do:

1. List the most recent tweets of Donald Trump's (or another Twitter user of interest) and predict if they are positive or negative.
2. Build a DataFrame of tweets, and add a column indicating the postive or negative predictions.
3. Plot the predicted sentiments in a time series against other factors to see if there might be some relationship, for example between positive tweets and the number of retweets or likes vs. negative tweets.

Add your own cells below with your own working.

When you're finished with exercise 16,

If you are running this notebook using Binder, choose **Save and Checkpoint** from the **File** menu, **rename** your notebook to add a hyphen and your initials to the notebook name e.g. `Ex16_Sentiment_Analysis-DJ`, then choose **Download as Notebook** and save it to your computer or USB stick.

If you are running this notebook on your own machine, choose **Save and Checkpoint** from the **File** menu, choose **Make a copy** from the **File** menu, then **rename** your notebook to add a hyphen and your initials to the notebook name e.g. rename from `Ex16_Sentiment_Analysis-Copy1` to `Ex16_Sentiment_Analysis-DJ`.